In [ ]:
import sys
sys.path.append('/Users/vdk/GeneveWork/Code/calibpipe-main-calibpipe/')
from astropy.io import ascii
import numpy as np
import pandas as pd
import matplotlib
import csv
import scipy
import matplotlib.pyplot as plt
from seaborn import histplot
import random
import copy
from os import path, makedirs
from astropy.coordinates import SkyCoord
from astropy.table import Table
import astropy.units as u
from ctapipe.containers import MuonEfficiencyContainer
from ctapipe.coordinates import CameraFrame, TelescopeFrame
from ctapipe.io import EventSource, EventSeeker
from ctapipe.core import TelescopeComponent
from ctapipe.core.traits import (
    List,
    Int,
    FloatTelescopeParameter,
    TelescopeParameter,
    Unicode,
)
from ctapipe.image.cleaning import tailcuts_clean
from ctapipe.image.muon import MuonRingFitter, MuonIntensityFitter
from traitlets.config import Config
from ctapipe.image import ImageProcessor
from ctapipe.image.muon import MuonProcessor
from ctapipe.visualization import CameraDisplay
from ctapipe.instrument import CameraGeometry
from ctapipe.calib import CameraCalibrator
from traitlets.config.loader import Config, FileConfigLoader, JSONFileConfigLoader
import pathlib
from calibpipe.throughput import MuonCalibrator
from traitlets.config import Config
from astropy.time import Time
from astropy.coordinates import EarthLocation,SkyCoord, AltAz 
import time
from ctapipe.image import number_of_island_sizes
from ctapipe.image import (
    hillas_parameters,
    leakage_parameters,
    concentration_parameters,
)
from ctapipe.image import timing_parameters
from ctapipe.image import number_of_islands
from ctapipe.image import camera_to_shower_coordinates
import ctapipe
from lstchain.io import replace_config, standard_config
import h5py
from astropy.io import fits
import json
import os
from pyirf import spectral,simulations,utils 

## Observations

In [ ]:
# Треба завантажити конфіг в івент соурс
filename = f'/Users/vdk/RealLST/LST-1.1.Run15085.0001.fits.fz'

int_cut = 0.3
min_pixel = 30

config_filename = '/Users/vdk/RealLST/typical_run_data/lstchain_standard_v0.10_heuristic_ff.json'
with open(config_filename) as json_file:
    config_data = json.load(json_file)

custom_config = Config(config_data)

muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})

muon_processor_config_2 = Config({"MuonProcessor":
                                {"ImageParameterQuery" :
                                 {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

cfg = Config({
    "source_config": {
        "EventSource" : {
            "allowed_tels": [1],"max_events": 9999}},
        "PointingSource":{
            "drive_report_path": '/Users/vdk/RealLST/data/DrivePosition_log_20231007.txt'},
        "LSTR0Corrections": {
          "calib_scale_high_gain":1.088,
          "calib_scale_low_gain":1.004,
          "drs4_pedestal_path": '/Users/vdk/RealLST/data/drs4_pedestal.Run14937.0000.h5',
          "calibration_path": '/Users/vdk/RealLST/data/calibration_filters_52.Run14938.0000.h5',
          "drs4_time_calibration_path": '/Users/vdk/RealLST/data/time_calibration.Run08349.0000.h5'
      }})

config = replace_config(standard_config, muon_processor_config)
config = replace_config(config, muon_processor_config_2)
config = replace_config(config, custom_config)


In [ ]:
source = EventSource(filename, config = Config(cfg))
event_iterator = iter(source)

In [ ]:
time_2 = []
for event in event_iterator:
    time_2.append(event.trigger.time.to_value(format = 'unix_tai'))

In [ ]:
len(time_2)

In [ ]:
time_2[-1] -  time_2[0]

In [ ]:
time_2[-1]

In [ ]:
time[-1]

In [ ]:
event.trigger.time.to_value(format = 'unix_tai')

In [ ]:
1696720155.4783056 - 1696720155.477298

## Simulations

In [ ]:
muons_energy = [1.203954219818115234e+00, 1.407119393348693848e+00, 3.442558348178863525e-01, 7.431100308895111084e-02, 5.539828166365623474e-02, 6.081026196479797363e-01, 1.953697204589843750e+01, 1.592359095811843872e-01]

w_muons = sum([energy**(-0.7) for energy in muons_energy])

with open('/Users/vdk/runs31-41_protons.txt', 'r') as fi:
    lines1 = fi.readlines()

with open('/Users/vdk/runs41-51_protons.txt', 'r') as fi:
    lines2 = fi.readlines()

splitted = lines1[0].split(' ')
protons_energy1 = [float(i) for i in splitted]
splitted = lines2[0].split(' ')
protons_energy2 = [float(i) for i in splitted]

protons_energy = protons_energy1 + protons_energy2

w_protons = sum([energy**(-0.7) for energy in protons_energy])
print("Weighted muons = ", w_muons)
print("Weighted protons = ", w_protons)

In [ ]:
Nshow = 20e4 * 10 * 500
E_min = 10
E_max = 103901
viewcone = 9
side = 1511.53
area = side**2
solid_angle = utils.cone_solid_angle(viewcone*u.deg).to_value()

# E^-2.7 from 10 to 10e5 GeV
energy_integral_true_spec = 0.01173 
# E^-2.7 from 10 to 103901 GeV
energy_integral_true_spec = 0.01172

# E^-0.7 from 10 to 10e5 GeV
energy_integral_weigthed_spec = 98.75 
# E^-0.7 from 10 to 103901 GeV
energy_integral_weigthed_spec = 99.97 

energy_integral_true_spec = 0.1


In [ ]:
Nshow

In [ ]:
def intensity(area, solid_angle, energy_integral):
    N_0 = 1.8e4
    return N_0 * area * solid_angle * 0.74 * energy_integral

In [ ]:
proton_rate = intensity(area=area, solid_angle = solid_angle, energy_integral=energy_integral_true_spec)
proton_number = Nshow * energy_integral_weigthed_spec
sim_time = proton_number / proton_rate
print(f"Simulated time {sim_time} sec")

In [ ]:
w_protons/sim_time

In [ ]:
(w_muons*6)/sim_time

In [ ]:
w_muons

In [ ]:
17.96/3719

In [ ]:
4.771929824561403/3719

In [ ]:

w_muons

In [ ]:
w_muons/elapsed_time

In [ ]:
166/elapsed_time

In [ ]:
266/elapsed_time

In [ ]:
116/9288

In [ ]:
166/1.42

In [ ]:
protons_energy[0]**(-0.7)

In [ ]:
Nshow / intensity

In [ ]:
def intensity(area, solid_angle):
    return 1.8 * 10000 * solid_angle * area * 0.74

In [ ]:
test = intensity(area, solid_angle)

In [ ]:
Nshow / (test * integral)

In [ ]:
472/1.47

In [ ]:
pyirf.__dir__()

In [ ]:
spectral.PDG_ALL_PARTICLE

In [ ]:
power_law = spectral.PowerLaw(normalization=1 * spectral.DIFFUSE_FLUX_UNIT,
        index=-2)

In [ ]:
simulated_events = simulations.SimulatedEventsInfo(n_showers=int(20e6),
                                                   energy_min=10 * u.GeV,
                                                   energy_max=100 * u.TeV,
                                                   max_impact=1500 * u.m,
                                                   spectral_index=-2,
                                                   viewcone=10 * u.deg)

energy_bins = np.geomspace(simulated_events.energy_min, simulated_events.energy_max, 300)

In [ ]:
#plt.hist(simulated_events.calculate_n_showers_per_energy(energy_bins), log = True)

sum(simulated_events.calculate_n_showers_per_energy(energy_bins))

In [ ]:
power_law.from_simulation(simulated_events, 5*u.second)

In [ ]:
power_law.integrate_cone(0*u.deg, 10*u.deg)

In [ ]:
spectral.calculate_event_weights(100*u.TeV, spectral.PDG_ALL_PARTICLE, power_law)

In [ ]:
utils.cone_solid_angle(10*u.deg)

In [ ]:
power_law = spectral.PowerLaw(normalization=1 * spectral.DIFFUSE_FLUX_UNIT,index=-2, e_ref= 1*  u.TeV)

w_protons = sum([spectral.calculate_event_weights(energy*u.TeV, spectral.PDG_ALL_PARTICLE, power_law) for energy in muons_energy])

In [ ]:
w_protons

In [ ]:
help(spectral.PowerLaw)